In [7]:
%matplotlib inline
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib.ticker
import seaborn as sns
%load_ext autoreload
%autoreload 2
pd.options.mode.chained_assignment = None

from settings import *
from glove import *
import re
import pickle
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import KFold
from textblob import TextBlob
from tweets_ops import *
from sklearn.metrics import confusion_matrix



The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [8]:
#alc_tob_caf use-case
# articles = pd.read_csv(cache_dir+'article_details_v5.tsv', sep='\t')

# articles = articles[articles.topics.apply(lambda x: (max(eval(x)[0]) != 0.062500) & (max(eval(x)[0]) == eval(x)[0][11]))]
# articles = articles[articles.entities.apply(lambda x: any(key in x for key in ['alcohol', 'tobacco', 'caffeine']))]

# articles.to_csv(cache_dir+'alc_tob_caf.tsv', sep='\t', index=None)

# aggregate_tweet_details(cache_dir+'diffusion_graph/pruned_graph_v3.tsv', cache_dir+'tweet_details_v2.tsv', cache_dir+'alc_tob_caf.tsv', cache_dir+'alc_tob_caf.tsv')

# articles = pd.read_csv(cache_dir+'alc_tob_caf.tsv', sep='\t')
# articles = articles[articles.replies.apply(lambda x: x != '[]')][['url', 'title']]

# df1 = pd.read_csv(cache_dir+'similarity_model/train_pairs_v1.tsv', sep='\t')
# df1 = df1[df1['related']]
# df2 = pd.read_csv(cache_dir+'similarity_model/test_pairs_v3.tsv', sep='\t')
# df2 = df2[df2['related']]
# df = pd.concat([df1, df2]).drop('related', axis=1)

# articles.merge(df, left_on='url', right_on='article').drop('url', axis=1).groupby(['article', 'title'])['paper'].apply(lambda x: '\n'.join(x)).reset_index().to_csv(cache_dir+'alc_tob_caf.tsv', sep='\t')

#crispr use-case
# articles = pd.read_csv(cache_dir+'article_details_v5.tsv', sep='\t')

# articles = articles[articles.entities.apply(lambda x: any(key in x for key in ['CRISPR']))]
# articles = articles[articles['url'].apply(lambda x: not x.startswith('http://www.coach-boursier.com'))]

# articles.to_csv(cache_dir+'crispr.tsv', sep='\t', index=None)

# aggregate_tweet_details(cache_dir+'diffusion_graph/pruned_graph_v3.tsv', cache_dir+'tweet_details_v2.tsv', cache_dir+'crispr.tsv', cache_dir+'crispr.tsv')

# articles = pd.read_csv(cache_dir+'crispr.tsv', sep='\t')

# df1 = pd.read_csv(cache_dir+'similarity_model/train_pairs_v1.tsv', sep='\t')
# df1 = df1[df1['related']]
# df2 = pd.read_csv(cache_dir+'similarity_model/test_pairs_v3.tsv', sep='\t')
# df2 = df2[df2['related']]
# df = pd.concat([df1, df2]).drop('related', axis=1)

# articles.merge(df, left_on='url', right_on='article').drop('url', axis=1).groupby(['article', 'title'])['paper'].apply(lambda x: '\n'.join(x)).reset_index().to_csv(cache_dir+'crispr.tsv', sep='\t')

In [9]:
# #one source articles
# df = pd.read_csv(cache_dir+'similarity_model/test_pairs_v3.tsv', sep='\t')

# df = pd.DataFrame(df.groupby(['article'])['related'].apply(list).reset_index()['related'])

# one_source = pd.read_csv(cache_dir+'similarity_model/train_pairs_v1.tsv', sep='\t').shape[0]
# articles_num = df.shape[0] + one_source

# one_source += df[df['related'].apply(lambda x: sum(x)==1)].shape[0]

# one_source/articles_num

In [10]:
# snopes = pd.read_csv('/home/psmeros/Dropbox/scilens/snopes-u.csv')
# articles = pd.read_csv(cache_dir+'article_details_v5.tsv', sep='\t')

# def get_snopes_articles(title, snopes_list, snopes_list_vec):
#     for sno, vec in zip(snopes_list, snopes_list_vec):
#         title_vec = sent2vec(title)
#         sim = cos_sim(vec, title_vec)
#         if sim > 0.8:
#             print('title:', title)
#             print('snopes:', sno)
#             print(sim)
#     return


# articles.apply(lambda x: get_snopes_articles(x['title'], snopes['claim'].tolist(), snopes['claim'].apply(sent2vec).tolist()), axis=1)

In [15]:
stance_data_dir = conf['aux_dir'] + 'small_files/stance/'

In [30]:
df = pd.concat([pd.read_csv(stance_data_dir+'train.tsv', sep='\t'),pd.read_csv(stance_data_dir+'train.tsv', sep='\t')]).reset_index(drop=True)
df = df[['Tweet', 'Stance']]
#df['Stance'] = df['Stance'].apply(lambda x: )
df

,Tweet,Stance
0,"@tedcruz And, #HandOverTheServer she wiped cle...",AGAINST
1,Hillary is our best choice if we truly want to...,FAVOR
2,@TheView I think our country is ready for a fe...,AGAINST
3,I just gave an unhealthy amount of my hard-ear...,AGAINST
4,@PortiaABoulger Thank you for adding me to you...,NONE
5,Hillary can not win. Here's hoping the Dems of...,AGAINST
6,Respect FOR the law and respect BY the law Yes...,NONE
7,I don't want to be appointed to an Ambassador ...,NONE
8,#StopHillary2016 @HillaryClinton if there was ...,AGAINST
9,@HillaryClinton End lawless #ClintonFoundation...,AGAINST


In [31]:
from nltk.sentiment.vader import SentimentIntensityAnalyzer
sid = SentimentIntensityAnalyzer()

In [35]:
df = df.join(df['Tweet'].apply(lambda x: sid.polarity_scores(x)).apply(pd.Series))

In [ ]:
df = pd.read_csv(cache_dir+'../corpus/stance_classification.csv').rename(columns={'what_do_you_believe_is_the_repliers_stance_position_towards_the_tweet': 'stance', 'what_do_you_believe_is_the_repliers_stance_position_towards_the_tweet:confidence': 'confidence'})
df = df[df.confidence>0.5][['full_text', 'reply', 'stance']]
df = df[df['stance']!='nr']

In [ ]:
tweets = pd.read_csv(cache_dir+'tweet_details_v2.tsv', sep='\t')
df = tweets[['full_text']]
df['reply'] = tweets['replies'].apply(lambda x: ' '.join(eval(x)))


In [ ]:
df['sim'] = df.apply(lambda x: cos_sim(sent2vec(x['full_text']), sent2vec(x['reply'])),axis=1)

In [ ]:
df['word_count'] = df['reply'].apply(lambda x: len((re.sub(' +',' ',re.sub(r'[^a-zA-Z0-9 ]', '', x))).strip().split(' ')))

In [ ]:
df['negation'] = df['reply'].apply(lambda x: any(n in x for n in [' no ', ' not ', 'n\'t ']))

In [ ]:
positive_words = open(conf['aux_dir'] + 'small_files/opinion/positive-words.txt', encoding='utf-8', errors='ignore').read().splitlines()
negative_words = open(conf['aux_dir'] + 'small_files/opinion/negative-words.txt', encoding='utf-8', errors='ignore').read().splitlines()
df['positive'] = df['reply'].apply(lambda x: sum(n in x for n in positive_words))
df['negative'] = df['reply'].apply(lambda x: sum(n in x for n in negative_words))

In [ ]:
df['length'] = df['reply'].apply(len)

In [ ]:
df['has_url'] = df['reply'].apply(lambda x: bool(re.search('http(s)?://', x)))

In [ ]:
df['quest_mark'] = df['reply'].apply(lambda x: x.count('?'))

In [ ]:
df['excl_mark'] = df['reply'].apply(lambda x: x.count('!'))

In [ ]:
df['reply_polarity'] =  df['reply'].apply(lambda x: TextBlob(x).sentiment.polarity)
df['reply_subjectivity'] =  df['reply'].apply(lambda x: TextBlob(x).sentiment.subjectivity)

In [ ]:
df['stance'] = df['stance'].apply(lambda x: 'con-quest' if x in ['con', 'quest'] else x).apply(lambda x: 'nr-com-sup' if x in ['nr', 'com', 'sup']  else x)

In [ ]:
X = np.array(df.drop(['full_text', 'reply', 'stance'], axis=1).values, dtype=np.float32)
y = df['stance'].values

In [ ]:
df['stance'].value_counts()

In [ ]:
cross_val = False
n_est = 100
m_dep = 10
fold = 2
if cross_val:
    kf = KFold(n_splits=fold, shuffle=True)
    score = 0.0
    for train_index, test_index in kf.split(X):
        X_train, X_test = X[train_index], X[test_index]
        X_train = (X_train - X_train.mean(axis=0)) / (X_train.std(axis=0)+1e-9)
        X_test = (X_test - X_test.mean(axis=0)) / (X_test.std(axis=0)+1e-9)
        
        y_train, y_test = y[train_index], y[test_index]
        classifier = RandomForestClassifier(n_estimators=n_est, max_depth=m_dep, n_jobs=-1)
        classifier.fit(X_train, y_train)

        score += classifier.score(X_test, y_test)
        conf_mat = confusion_matrix(y_test, classifier.predict(X_test))
        conf_mat = conf_mat.astype('float') / conf_mat.sum(axis=1)[:, np.newaxis]
        print(conf_mat)
    print('Score:', score/fold)
else:
    classifier = RandomForestClassifier(n_estimators=n_est, max_depth=m_dep, n_jobs=-1)
    X = (X - X.mean(axis=0)) / (X.std(axis=0)+1e-9)
    classifier.fit(X, y)
    pickle.dump(classifier, open(cache_dir+'stance.pkl', 'wb'))    

In [ ]:
X = np.array(df.drop(['full_text', 'reply'], axis=1).values, dtype=np.float32)
X = (X - X.mean(axis=0)) / (X.std(axis=0)+1e-9)
classifier = pickle.load(open(cache_dir+'stance.pkl', 'rb'))
tweets['stance']=classifier.predict_proba(X)[:,1]


In [ ]:
tweets['stance']= tweets.apply(lambda x: 0 if x.replies_num==0 else x.stance, axis=1)

In [ ]:
tweets.to_csv(cache_dir+'tweet_details_v3.tsv', sep='\t', index=None)